## **Assistants API - Function Calling**

An assistant is a purpose-built AI that has specific instructions, leverages extra knowledge, and can call models and tools to perform tasks.

https://platform.openai.com/docs/assistants/how-it-works/objects

In [1]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import json

_ = load_dotenv(find_dotenv())
client: OpenAI = OpenAI()

### **Function Calling**

Similar to the Chat Completions API, the Assistants API supports function calling. Function calling allows you to describe functions to the Assistants and have it intelligently return the functions that need to be called along with their arguments. The Assistants API will pause execution during a Run when it invokes functions, and you can supply the results of the function call back to continue the Run execution.

### **Step 0: Define Functions**

In [2]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def getCurrentWeather(location: str, unit:str="fahrenheit") -> str | dict | None: 
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    
def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

### **Step 1: Create an Assistant and register/report your functions**

In [3]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json())) 

In [4]:
from openai.types.beta.assistant import Assistant

assistant: Assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo-1106",
    tools=[
        {
            "type":"function",
            "function": {
                "name":"getCurrentWeather",
                "description": "Get the weather in location",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "location":{"type":"string", "description": "The city and state e.g. San Francisco, CA"},
                        "unit":{"type":"string", "enum": ["c", "f"]},
                    },
                    "required": ["location"]
                }
            }
        },
        {
            "type": "function",
            "function": {
            "name": "getNickname",
            "description": "Get the nickname of a city",
            "parameters": {
                    "type": "object",
                    "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                    },
                    "required": ["location"]
                }
            } 
        }
    ]
)

### **Step 2: Create a Thread**

In [5]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

print(thread)

Thread(id='thread_GtNtop1eWQugTnwrSHIm8ngl', created_at=1704311620, metadata={}, object='thread')


In [6]:
from openai.types.beta.threads.thread_message import ThreadMessage

message: ThreadMessage = client.beta.threads.messages.create(
    content="How is the weather in Los Angles?",
    thread_id=thread.id,
    role="user"
)

In [7]:
dict(message)

{'id': 'msg_OKdWlTXadBVxwzLcs9MrkbUw',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='How is the weather in Los Angles?'), type='text')],
 'created_at': 1704311620,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_GtNtop1eWQugTnwrSHIm8ngl'}

### **Step 4: Run the Assistant**

In [8]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [9]:
dict(run)

{'id': 'run_24QXLWPu2DLu6xo2l9gGW1sv',
 'assistant_id': 'asst_LtB3aQiQcthXGkKdruePlFxY',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704311621,
 'expires_at': 1704312221,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_GtNtop1eWQugTnwrSHIm8ngl',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object

### **Run Life Cycle**

![Alt Text](diagram.png)

In [10]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname,
}

In [11]:
thread.id

'thread_GtNtop1eWQugTnwrSHIm8ngl'

### **Step 5: Polling for Updates and Calling Functions**

In [12]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break




in_progress .....
Run is queued. Waiting...
requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_nGai6nE8DGAivXVR8AHk097R',
    'function': {'arguments': '{"location":"Los Angeles, CA","unit":"c"}',
     'name': 'getCurrentWeather'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
<function getCurrentWeather at 0x109e59c60> True ================================================================
[{'tool_call_id': 'call_nGai6nE8DGAivXVR8AHk097R', 'output': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'}] >>>>>
in_progress .....
Run is queued. Waiting...
completed .....
completed...........logic
Assistant: The current temperature in Los Angeles, CA is 72°F.

User: How is the weather in Los Angles?

